In [3]:
# Credit to Arpitha and Savi

# p(psuedocounts)
# i(iterations)
# k(kmerlength)

# inputfile
# >header1
# seq (50 bases) (6 sequences)
# header2
# seq...
# randomly grab a motif in the sequence depending on the input(kmerlength) hardcode input commandline
# put all the random motifs in a 'motif matrix'
# count matrix will have the counts of each base in each column
# always use pseudocount which adds 1 or whatever the number of pseudocounts is to each base in each column for the count matrix
# profile matrix will have the proportion of number of each base / the total number of all the bases.
# to calculate the score we need the total proportions of each sequence. If there are 300 bases in total then the Q(A) = total number of As in genome/300 = Q
# Then to calculate the score input the profile matrix P(X) and total proportion Q(X).
# The relative entropy sequence is the seq with the largest percentage of base in the sequence
# Get the sequence most probable based on the profile matrix
# To get the relative entropy sequence you get the biggest probability of each column
# double loop. inner loop that goes through the motif matrix to the count matrix to the profile matrix and the outer loop is to start all over.
# The outer loop has a specific amount of iterations.
# You want the smaller score when comparing scores after your done with the iterations each time.
# one iteration is done when the relative entropy sequence and score stops changing. After that you then have to find every possible kmer and compare
# probability of each base in the kmer (based on the profile matrix) and multiply them together and the highest value is the one you keep in the motif matrix.
# put getting the data and running the iterations in main


class CommandLine() :
    '''
    Handle the command line, usage and help requests.
    CommandLine uses argparse, now standard in 2.7 and beyond.
    it implements a standard command line argument parser with various argument options,
    a standard usage and help, and an error termination mechanism do-usage_and_die.
    attributes:
    all arguments received from the commandline using .add_argument will be
    avalable within the .args attribute of object instantiated from CommandLine.
    For example, if myCommandLine is an object of the class, and requiredbool was
    set as an option using add_argument, then myCommandLine.args.requiredbool will
    name that option.
    '''

    def __init__(self, inOpts=None) :
        '''
        CommandLine constructor.
        Implements a parser to interpret the command line argv string using argparse.
        '''
        import argparse
        self.parser = argparse.ArgumentParser(
            description='Program prolog - a brief description of what this thing does',
            epilog='Program epilog - some other stuff you feel compelled to say',
            add_help=True,  # default is True
            prefix_chars='-',
            usage='%(prog)s [options] -option1[default] <input >output'
        )
        self.parser.add_argument('-i', '--iterations', nargs='?', type=int, default=1, action='store',
                                 help='iterations')
        self.parser.add_argument('-p', '--pseudocount', nargs='?', type=float, default=1, action='store',
                                 help='pseudocount')
        self.parser.add_argument('-k', '--kmer', nargs='?', type=int, default=.01, action='store',
                                 help='kmer length')
        self.parser.add_argument('-v', '--version', action='version', version='%(prog)s 0.1')
        if inOpts is None :
            self.args = self.parser.parse_args()
        else :
            self.args = self.parser.parse_args(inOpts)

import sys


class FastAreader :
    '''FastAreader program provided by the professor. '''

    def __init__(self, fname='') :
        '''contructor: saves attribute fname '''

        self.fname = fname
        self.fileH = None

    def doOpen(self) :
        if self.fname == '' :
            return sys.stdin
        else :
            return open(self.fname)

    def readFasta(self) :

        header = ''
        sequence = ''

        with self.doOpen() as self.fileH :

            header = ''
            sequence = ''

            # skip to first fasta header
            line = self.fileH.readline()
            while not line.startswith('>') :
                line = self.fileH.readline()
            header = line[1 :].rstrip()

            for line in self.fileH :
                if line.startswith('>') :
                    yield header, sequence
                    header = line[1 :].rstrip()
                    sequence = ''
                else :
                    sequence += ''.join(line.rstrip().split()).upper()

        yield header, sequence

import math
import random
import numpy as np

class CRISPR:
    def __init__(self, iter, PC, kmers):
        '''
        Initializes the iterations, pseudocount and kmer length.
        Creates an empty list for the motif, count, and profile matrix.
        Initializes the probability for each base
        '''
        self.iter = int(iter)
        self.PC = int(PC)
        self.kmers = kmers
        self.motifMatrix = []
        self.countMatrix = []
        self.profileMatrix = []
        self.genome = []
        self.conSeq = None
        self.probability_A = 0
        self.probability_C = 0
        self.probability_T = 0
        self.probability_G = 0

    def randMotif(self):
        '''
        randomly selects a motif on the genome and adds to the motif list. From Chris OH
        '''
        motifList = []
        for seq in self.genome:
            start = random.randint(0, len(seq) - self.kmers) # picks a random position in the genome to start the kmer
            end = int(start) + self.kmers
            motif = list(seq[start:end])
            motifList.append(motif) #add the kmer to a list
        self.motifMatrix = np.array(motifList) #make list an array
        return self.motifMatrix
    def getIterations(self):
        '''
        returns the number of iterations
        '''
        return self.iter

    def arrayMotif(self, nucList):
        '''
        Converts the motif matrix into an array so I can later use the array to count the columns
        '''
        motifList = []
        for i in range(len(nucList)):
            code = list(nucList[i])
            motifList.append(code)
        self.motifMatrix = np.array(motifList)
        return self.motifMatrix

    def countsMatrix(self):
        '''
        counts the number of each base in each column, plus the pseudocount and puts it in the count matrix
        '''
        A_list = []; T_list = []; C_list = []; G_list = []
        for val in range(self.kmers):
            col = self.motifMatrix[:,val] # gets the columns in the motif matrix
            countA = sum(np.char.count(col, 'A'), self.PC) #adds the number of bases in each column plus the pseducount
            countT = sum(np.char.count(col, 'T'), self.PC)
            countC = sum(np.char.count(col, 'C'), self.PC)
            countG = sum(np.char.count(col, 'G'), self.PC)
            A_list.append(countA)
            T_list.append(countT)
            C_list.append(countC)
            G_list.append(countG)
        count_list = [A_list, T_list, G_list, C_list] #puts the count of each base into a list
        self.countMatrix = np.array(count_list) #makes list an array
        return self.countMatrix

    def profilesMatrix(self):
        '''
        calculates the proportion of each base in each column and puts it in the profile matrix
        '''
        A_prob = []; T_prob = []; C_prob = []; G_prob = []
        for val in range(self.kmers):
            col = self.countMatrix[:, val]
            x = sum(col)
            A_prob.append(col[0] / x) # gets the proportion of each base in each column over the total nucleotides
            T_prob.append(col[1] / x)
            C_prob.append(col[2] / x)
            G_prob.append(col[3] / x)
        prob_list = [A_prob, T_prob, C_prob, G_prob] #puts the proportion of each base into a list
        self.profileMatrix = np.array(prob_list) #make the list an array
        return (self.profileMatrix)

    def consensusSeq(self):
        '''
        Finds the consensus sequence by iterating through the kmer length
        and profile matrix. Finds the max probability in each column in the profile
        matrix.
        '''
        seq = []
        for val in range(self.kmers):
            col = self.profileMatrix[:, val]
            base = max(col)
            if base == col[0]:  #adds the highest probability of each base to the consensus sequence
                seq.append('A')
            elif base == col[1]:
                seq.append('T')
            elif base == col[2]:
                seq.append('C')
            else:
                seq.append('G')
        self.conSeq = ''.join(seq)
        return self.conSeq

    def relativeE(self):
        '''
        calculates the relative entropy score based on the profile matrix and the total number of bases in the genome.
        '''
        countRE = 0
        for val in range(self.kmers):
            col = self.profileMatrix[:, val]
            probA = col[0]; probT = col[1]; probC = col[2]; probG = col[3]

            aProb = probA * math.log2(probA / self.probability_A) # calculates the entropy score for each base
            tProb = probT * math.log2(probT / self.probability_T)
            cProb = probC * math.log2(probC / self.probability_C)
            gProb = probG * math.log2(probG / self.probability_G)
            scoreRE = (aProb + tProb + cProb + gProb) # adds up all the scores
            countRE += scoreRE
        return countRE

    def motifProb(self, seq):
        '''
        Finds the highest probability out of all the possible kmers in the sequence
        '''
        probDict = {}
        start = 0
        end = self.kmers - 1
        while (end < len(seq)):
            motifSeq = seq[start:end + 1]
            motifProb = 1
            baseNum = 0
            for nuc in motifSeq:
                if nuc == 'A':
                    motifProb = motifProb * self.profileMatrix[0][baseNum]
                elif nuc == 'T':
                    motifProb = motifProb * self.profileMatrix[1][baseNum]
                elif nuc == 'C':
                    motifProb = motifProb * self.profileMatrix[2][baseNum]
                else:
                    motifProb = motifProb * self.profileMatrix[3][baseNum]
            probDict[motifSeq] = motifProb
            start += 1
            end += 1
        finalMotif = max(probDict, key = lambda x: probDict[x])
        return finalMotif

    def calcProb(self, sourceReader):
        '''
        Calculates every nucleotide in the file and all the nucleotides in the final to get the
        probability of each base.
        '''
        countA = 0; countT = 0; countC = 0; countG = 0; genomeCount = 0
        bases = ['A', 'T', 'C', 'G']
        for head, seq in sourceReader.readFasta():
            seq = ''.join(filter(lambda x: x in bases, seq))
            countA += seq.count('A')
            countT += seq.count('T')
            countC += seq.count('C')
            countG += seq.count('G')
            genomeCount += len(seq)
            self.genome.append(seq)

        countA = countA + self.PC
        countT = countT + self.PC
        countC = countC + self.PC
        countG = countG + self.PC
        genomeCount = genomeCount + (4 * self.PC)

        self.probability_A = countA / genomeCount
        self.probability_T = countT / genomeCount
        self.probability_C = countC / genomeCount
        self.probability_G = countG / genomeCount


def main(inFile=None, options=None):
    '''
    Gets the data runs the iterations. Check to see if the score is better or not
    '''
    cl = CommandLine(options)
    sourceReader = FastAreader(inFile)  # setup the Fasta reader Object
    thisCRISPR = CRISPR(cl.args.iterations, cl.args.pseudocount, cl.args.kmer)
    thisCRISPR.calcProb(sourceReader)
    bases = ['A', 'T', 'C', 'G']

    bestMotif = None
    bestScore = math.inf
    for i in range(thisCRISPR.getIterations()):
        prevScore = -math.inf
        thisCRISPR.randMotif() # Get the initial set of random motifs
        thisCRISPR.countsMatrix()
        thisCRISPR.profilesMatrix()
        conSeq = thisCRISPR.consensusSeq()
        relE = thisCRISPR.relativeE()
        while (prevScore != relE):  #Loop through until the score stops changing
            var = relE
            prevScore = var
            x = []
            for head, seq in sourceReader.readFasta():
                seq = ''.join(filter(lambda x : x in bases, seq))
                newM = thisCRISPR.motifProb(seq)
                x.append(newM)
            thisCRISPR.arrayMotif(x)
            thisCRISPR.countsMatrix()
            thisCRISPR.profilesMatrix()
            conSeq = thisCRISPR.consensusSeq()
            relE = thisCRISPR.relativeE()
        if bestMotif is None and bestScore == math.inf: #if statement comparing the motif and score
            bestMotif = conSeq
            bestScore = relE
        else:
            if bestScore < relE:  # if the score stops changing then that is the new best score
                bestMotif = conSeq
                bestScore = relE
    print("Consensus Sequence: {bestMotif}".format(bestMotif=bestMotif))
    print("Best Score: {bestScore}".format(bestScore=bestScore))


if __name__ == "__main__" :
    main(inFile="input.fa", options=["-i=1000", "-p=1", "-k=13"])

main()

Consensus Sequence: AAAAAGTTAAAAA
Best Score: 9.190159789522177


usage: ipykernel_launcher.py [options] -option1[default] <input >output
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-34bb2498-fb5b-44ec-a1af-8e254e347d6a.json


SystemExit: ignored

Inspections:

Savi inspection notes:
        self.kmers = kmers should be int(kmers) too 
        change defualt to type = int in command line class, should be helpful
        def consensusSequence(self) has if statements that check maxprobability against T twice; sub one of those if statements for G
        add counter to def relativeE method... important for the double summation
        in your motifProb, make sure you have an else statement; if you have an elif, it seems like you will have an extra nuc that is not accounted for but there is not another
        you don't need the lowercase nucs because FastaReader already changes it to upper()

Jonny inspection notes:
  I think the algorithim in the main is good but could be more efficient by using a while and a for instead of a triple nested loop. Having a separte method for iterating through the iterations. I also think you could add more in line comments.

Arsalan inspection notes:
  The profile matrix is based on the counts. He used numpy array to hold the sequences in the motif matrix. The iterations was done in the main method.

Inspection results:
  I believe putting the iteration in the main method was the best and most efficient way to do so. I think next time I should add more comments to explain the code. Also some methods could probably be combined in one which could also decrease run time. My output is not completely correct and I keep getting a type error every time I run that I'm not sure how to fix. I was getting this same error last week too.
